In [1]:
from pytrends.request import TrendReq
import pandas as pd
import sqlite3
import numpy as np

In [2]:
#obtendo as tabelas de indicadores
conn =sqlite3.connect("indicadores.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
brasilDf=pd.read_sql_query("select * from brazil_df",conn)
conn.close()

[('imedreg_df',), ('cities_df',), ('states_df',), ('regions_df',), ('brazil_df',)]


In [3]:
#lendo o arquivo com as keywords
keywords=pd.read_csv('Queries categorizadas.csv')
keywords

,Unnamed: 0,referencia,query,Categoria,Subcategoria
0,1,erupção cutânea,sintomas do covid,Sintoma,geral
1,2,erupção cutânea,covid-19,Vírus ou Doença,geral
2,3,erupção cutânea,sintomas do covid 19,Sintoma,geral
3,4,dor de garganta,dor de garganta,Sintoma,gripais
4,5,dor de cabeça,dor de cabeça,Sintoma,geral
...,...,...,...,...,...
120,121,respirador,respirador hospitalar,Medicação e Tratamento,respiradores
121,122,medicamento,medicamento coronavírus,Medicação e Tratamento,geral
122,123,medicamento,medicamento corona,Medicação e Tratamento,geral
123,124,estou com covid,estou com covid,Contraiui Covid,NaN


In [4]:
#gerando a lista de keywords
kw_list=[]
for i in range(len(keywords)):
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace('[','')
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace(']','')
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace('\'','')
    kw_list.append(keywords['query'].iloc[i])

C:\Users\msart\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
#como pode ser visto,a ultima semana totalmente em 2020 termina no dia 26 de dezembro e a primeira com dados
#começa dia 23 de fevereiro
print(brasilDf['week_last_day'].head())
print(brasilDf['week_last_day'].tail())

0    2020-02-29
1    2020-03-07
2    2020-03-14
3    2020-03-21
4    2020-03-28
Name: week_last_day, dtype: object
42    2020-12-19
43    2020-12-26
44    2021-01-02
45    2021-01-09
46    2021-01-16
Name: week_last_day, dtype: object


In [6]:
pytrends = TrendReq(hl='en-US', tz=360)

In [7]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos ao brasil
df=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR', gprop='')
    aux=pytrends.interest_over_time()
    df[kw_list[i]]=aux[kw_list[i]]

In [8]:
df.to_csv('SeriesKeywords.csv')

In [10]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos a MG
dfMg=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR-MG', gprop='')
    aux=pytrends.interest_over_time()
    #caso a consulta não consiga resultado, criará uma coluna de NAN
    if not aux.empty:
        dfMg[kw_list[i]]=aux[kw_list[i]]
    else:
        dfMg[kw_list[i]]=np.nan

In [11]:
dfMg.to_csv('SeriesKeywordsMg.csv')